# HRT Theory: From Paper to Implementation

This notebook demonstrates how the theoretical foundations from **"HLLSet Theory: Contextual Anti-Sets and the Selection Principle"** are implemented in the HRT (Hash Relational Tensor) framework.

## Theoretical Foundations

The paper establishes:
1. **HLLSet = (H, φ, τ, ρ)** - Contextual anti-sets with thresholds
2. **Bell State Similarity (BSS)** - Morphism conditions
3. **ε-isomorphism** - Structural entanglement
4. **Noether Current** - Conservation laws
5. **Evolution H(t+1) = [H(t) \\ D] ∪ N** - Kinematic dynamics
6. **Contextual Selection** - Contexts actively select elements

In [ ]:
import sys
sys.path.insert(0, '/home/alexmy/SGS/SGS_lib/hllset_manifold')

from core.hrt import (
    HRTConfig, HRT, HRTEvolution,
    BasicHLLSet, AdjacencyMatrix, HLLSetLattice,
    NoetherCurrent, EvolutionTriple, ContextualSelection
)
from core.kernel import Kernel
from core.hllset import HLLSet

kernel = Kernel()
print("✓ HRT framework loaded")

## 1. HLLSet Configuration with τ and ρ Thresholds

**From paper Section 2.1:**
- τ (tau): Inclusion tolerance threshold
- ρ (rho): Exclusion intolerance threshold
- Constraint: 0 ≤ ρ < τ ≤ 1

In [ ]:
# Create config with BSS thresholds
config = HRTConfig(
    p_bits=8,      # Smaller for demo
    h_bits=16,
    tau=0.7,       # Morphism requires 70% similarity
    rho=0.3,       # and at most 30% exclusion
    epsilon=0.1    # ε-isomorphism tolerance
)

print(f"Configuration:")
print(f"  Dimension: {config.dimension}")
print(f"  τ (inclusion): {config.tau}")
print(f"  ρ (exclusion): {config.rho}")
print(f"  ε (isomorphism): {config.epsilon}")
print(f"\nConstraint satisfied: 0 ≤ {config.rho} < {config.tau} ≤ 1 ✓")

## 2. Bell State Similarity (BSS) and Morphisms

**From paper Section 2.2:**

$$\text{BSS}_\tau(A \to B) = \frac{|A \cap B|}{|B|}$$

$$\text{BSS}_\rho(A \to B) = \frac{|A \setminus B|}{|B|}$$

A morphism $f: A \to B$ exists iff:
$$\text{BSS}_\tau(A \to B) \geq \tau \quad \text{and} \quad \text{BSS}_\rho(A \to B) \leq \rho$$

In [ ]:
# Create two HLLSets with different overlap
hllset_a = kernel.absorb(["apple", "banana", "cherry", "date"])
hllset_b = kernel.absorb(["banana", "cherry", "elderberry"])

# Create basic HLLSets
basic_a = BasicHLLSet(index=0, is_row=True, hllset=hllset_a, config=config)
basic_b = BasicHLLSet(index=1, is_row=True, hllset=hllset_b, config=config)

# Compute BSS
bss_tau = basic_a.bss_tau(basic_b)
bss_rho = basic_a.bss_rho(basic_b)
has_morphism = basic_a.has_morphism_to(basic_b)

print(f"Basic HLLSet A: {basic_a.name}")
print(f"  Cardinality: ~{int(hllset_a.cardinality())}")
print(f"\nBasic HLLSet B: {basic_b.name}")
print(f"  Cardinality: ~{int(hllset_b.cardinality())}")
print(f"\nBSS Analysis:")
print(f"  BSS_τ(A → B) = {bss_tau:.3f}")
print(f"  BSS_ρ(A → B) = {bss_rho:.3f}")
print(f"\nMorphism f: A → B exists? {has_morphism}")
print(f"  (requires BSS_τ ≥ {config.tau} and BSS_ρ ≤ {config.rho})")

## 3. ε-Isomorphism and Entanglement

**From paper Definition 4.1:**

Two lattices are ε-isomorphic if:
$$|\text{BSS}(A, B) - \text{BSS}(\phi(A), \phi(B))| \leq \epsilon$$

**From paper Theorem 4.2:**

$$P(\text{not } \epsilon\text{-isomorphic}) \leq \min\left(1, n^2 \cdot \left( \frac{d^2}{2^m} + e^{-\epsilon^2 d / 2} \right) \right)$$

In [ ]:
# Create two HRTs from similar data
hrt1 = HRT.empty(config)
hrt2 = HRT.empty(config)

# Ingest similar data
data1 = {"sensor_A": {"red", "green", "blue"}, "sensor_B": {"hot", "cold"}}
data2 = {"sensor_C": {"crimson", "emerald", "azure"}, "sensor_D": {"warm", "cool"}}

hrt1_ingested = HRT.from_perceptrons(data1, kernel, config)
hrt2_ingested = HRT.from_perceptrons(data2, kernel, config)

# Merge with empty to get actual HRTs
hrt1_merged = hrt1_ingested.merge(hrt1, kernel)
hrt2_merged = hrt2_ingested.merge(hrt2, kernel)

# Check entanglement
is_entangled, prob_failure = hrt1_merged.check_entanglement(hrt2_merged)

print(f"HRT 1: {hrt1_merged}")
print(f"HRT 2: {hrt2_merged}")
print(f"\nEntanglement Analysis:")
print(f"  ε-isomorphic: {is_entangled}")
print(f"  Failure probability: {prob_failure:.2e}")
print(f"  Entanglement confidence: {(1-prob_failure)*100:.4f}%")

## 4. Noether Current and Conservation

**From paper Section 6.2:**

$$J_{uv}(p) = p[u] \cdot (Ap)[v] - p[v] \cdot (A^T p)[u]$$

Total flux is conserved:
$$\frac{d\Phi}{dt} = 0$$

This monitors system health - drift indicates hash collisions or errors.

In [ ]:
# Create evolution manager
evolution = HRTEvolution(config)

# Ingest and evolve
perceptron_data = {
    "visual": {"red", "green", "blue"},
    "audio": {"low", "mid", "high"},
    "tactile": {"soft", "hard"}
}

evolution.ingest(perceptron_data, kernel)

# Mock commit function
def commit_fn(hrt):
    return hrt.name

evolution.evolve(kernel, commit_fn)

current = evolution.get_current()

print(f"Current HRT: {current}")
print(f"\nNoether Current:")
if current.noether_current:
    print(f"  Flux Φ = {current.noether_current.flux:.9f}")
    print(f"  Step: {current.noether_current.step_number}")
    print(f"\nConservation Health:")
    print(f"  {current.conservation_health()}")

## 5. Evolution Triple (D, R, N)

**From paper Section 5.1:**

$$H(t+1) = [H(t) \setminus D] \cup N$$

where:
- **D**: Deleted information (natural decay)
- **R**: Retained information (H(t) \\ D)
- **N**: New information (patterns to add)

**Conservation:** When |N| = |D|, evolution is conservative (ΔQ = 0)

In [ ]:
if current.evolution_triple:
    evo = current.evolution_triple
    print(f"Evolution Triple Analysis:")
    print(f"  D (Deleted): {len(evo.deleted)} HLLSets")
    print(f"  R (Retained): {len(evo.retained)} HLLSets")
    print(f"  N (New): {len(evo.new)} HLLSets")
    print(f"\nCardinality Change:")
    print(f"  ΔQ = |N| - |D| = {evo.cardinality_change}")
    print(f"  Conservative? {evo.is_conservative}")
    print(f"\nInterpretation:")
    if evo.cardinality_change > 0:
        print(f"  System is GROWING (learning new patterns)")
    elif evo.cardinality_change < 0:
        print(f"  System is CONTRACTING (forgetting patterns)")
    else:
        print(f"  System is STABLE (balanced evolution)")

## 6. Contextual Selection Principle

**From paper Section 7.1:**

The fundamental inversion: **contexts actively select compatible elements**.

$$S_C: \mathcal{U} \to \{0,1\}$$
$$S_C(x) = 1 \iff \text{BSS}(F_C, F_x) \geq \tau_C \text{ and exclusion}(F_C, F_x) \leq \rho_C$$

This explains:
- Quantum measurement (context = experimental setup)
- Biological evolution (context = ecological niche)
- Consciousness (context = self-selecting awareness)

In [ ]:
# Apply contextual selection
context_index = 2
selection = current.contextual_select(context_index=context_index, is_row=True)

print(f"Contextual Selection Analysis:")
print(f"\nContext: row basic HLLSet r_{context_index}")
print(f"  Context hash: {selection.context_hash[:16]}...")
print(f"\nSelection Criteria:")
print(f"  τ threshold: {selection.tau_threshold}")
print(f"  ρ threshold: {selection.rho_threshold}")
print(f"\nResults:")
print(f"  Selected: {len(selection.selected_hashes)} elements")
print(f"  Selection power: {selection.selection_power:.3f}")
print(f"\nInterpretation:")
print(f"  The context ACTIVELY SELECTED {len(selection.selected_hashes)} compatible elements")
print(f"  This is not passive membership - the context precedes and determines content!")

## 7. Time-Reversible Dynamics

**From paper Section 6.1:**

Forward projection:
$$\vec{p}_{\text{forward}} = \text{normalize}(A \cdot \vec{p})$$

Retro-cast (time reversal):
$$\vec{p}_{\text{retro}} = \text{normalize}(A^T \cdot \vec{p})$$

This enables:
- Future prediction from current state
- Past reconstruction from current state
- Conservation law verification

In [ ]:
# Project future and past
current_state = [2, 3, 4]  # Some column indices

future = current.project_future(current_state)
past = current.project_past(current_state)

print(f"Time-Reversible Projections:")
print(f"\nCurrent state: columns {current_state}")
print(f"\nFuture projection (A·p):")
print(f"  Shape: {future.shape}")
print(f"  Non-zero: {(future.data != 0).sum().item()}")
print(f"  Hash: {future.name[:16]}...")
print(f"\nPast reconstruction (A^T·p):")
print(f"  Shape: {past.shape}")
print(f"  Non-zero: {(past.data != 0).sum().item()}")
print(f"  Hash: {past.name[:16]}...")
print(f"\nTime symmetry: Both directions preserve information!")

## 8. Category Theory Structure

**From paper Section 3:**

Category **HLL**:
- **Objects**: HLLSets with (H, φ, τ, ρ)
- **Morphisms**: BSS-defined relations satisfying threshold conditions
- **Composition**: g ∘ f when conditions propagate
- **Identity**: 1_A with BSS_τ = 1, BSS_ρ = 0

**Karoubi Equivalence (Theorem 3.2):**
HLLSet category ≃ Karoubi completion of idempotent hash functions

In [ ]:
# Demonstrate category structure
lattice = current.lattice

print(f"Category HLL Structure:")
print(f"\nObjects (Basic HLLSets):")
print(f"  Row objects: {len(lattice.row_basic)}")
print(f"  Column objects: {len(lattice.col_basic)}")
print(f"  Total: {len(lattice.row_basic) + len(lattice.col_basic)}")

# Sample morphisms
print(f"\nMorphisms (sample):")
morphism_count = 0
for i in range(min(5, len(lattice.row_basic))):
    for j in range(min(5, len(lattice.row_basic))):
        if i != j and lattice.row_basic[i].has_morphism_to(lattice.row_basic[j]):
            morphism_count += 1
            if morphism_count <= 3:
                print(f"  f: {lattice.row_basic[i].name} → {lattice.row_basic[j].name}")

print(f"\nIdentity morphisms:")
print(f"  1_A: A → A exists for all A (trivially)")
print(f"  BSS_τ(A,A) = 1.0 (perfect self-similarity)")
print(f"  BSS_ρ(A,A) = 0.0 (no self-exclusion)")

print(f"\nCategory Properties:")
print(f"  ✓ Objects: HLLSets with (H, φ, τ={config.tau}, ρ={config.rho})")
print(f"  ✓ Morphisms: Defined by BSS thresholds")
print(f"  ✓ Composition: Transitive BSS preservation")
print(f"  ✓ Identity: Self-morphisms with BSS=1")

## Summary: From Theory to Practice

This notebook demonstrates how the theoretical paper is faithfully implemented:

1. **✓ BSS with τ/ρ thresholds** - Morphism conditions
2. **✓ ε-isomorphism** - Entanglement detection with probability bounds
3. **✓ Noether current** - Conservation monitoring (Φ ≈ 0)
4. **✓ Evolution triple (D,R,N)** - Explicit kinematic dynamics
5. **✓ Contextual selection** - Context-driven element selection
6. **✓ Time reversibility** - Forward/retro projections
7. **✓ Category theory** - Objects, morphisms, composition, identity

The HRT framework is not just a data structure - it's a complete implementation of the **Contextual Anti-Set** paradigm where:
- **Context precedes content**
- **Relationships are more real than relata**
- **Entanglement is the norm**
- **Information is conserved**